# Team member:
## Mengyao Li
## Feng Tian

In [16]:
import yfinance as yf
import numpy as np
import cvxpy as cp
import pandas as pd
from scipy.optimize import minimize

# Holding Ticker from XLG (Invesco S&P 500 Top 50 ETF)

In [17]:
tickers = ['MSFT','AAPL','NVDA','AMZN','GOOGL','META','GOOG','BRK-B','AVGO','LLY','JPM','XOM','TSLA','UNH','V','MA','PG','JNJ','HD',
'MRK','COST','CVX','ABBV','CRM','BAC','WMT','AMD','NFLX','PEP','KO','TMO','ADBE','WFC','LIN','DIS','MCD','CSCO','ACN',
'ORCL','ABT','QCOM','VZ','DHR','TXN','CMCSA','PM','PFE','NEE','INTC','NKE','BMY']

# Level 1

In [18]:
start_date, end_date = '2019-03-31', '2024-03-31'
data = {}
for ticker in tickers:
  data[ticker] = yf.download(ticker, start=start_date, end=end_date, progress=False)['Close']
df = pd.DataFrame(data)
df_returns = df.pct_change()
df_returns.dropna(inplace = True)
weights = np.array([1/len(tickers)] * len(tickers))
portfolio_returns = df_returns.dot(weights)
spy_returns = yf.download('SPY', start_date, end_date, progress=False)['Adj Close'].pct_change()
spy_returns.dropna(inplace = True)

In [19]:
def portfolio_stats(returns):
  annualized_r = ((returns + 1).prod()) ** (1/5)  - 1
  annualized_v = returns.std() * np.sqrt(252)
  var_95 = np.percentile(returns, 5)
  cvar_95 = returns[returns <= var_95].mean()

  cumulative_returns = (returns + 1).cumprod()
  peak = cumulative_returns.expanding(min_periods=1).max()
  drawdown = (cumulative_returns - peak) / peak
  max_drawdown = drawdown.min()
  return annualized_r, annualized_v, var_95, cvar_95, max_drawdown

annualized_r_p, annualized_v_p, var_95_p, cvar_95_p, max_drawdown_p = portfolio_stats(portfolio_returns)
annualized_r_s, annualized_v_s, var_95_s, cvar_95_s, max_drawdown_s = portfolio_stats(spy_returns)
performance = pd.DataFrame({'Equal Weighted': [annualized_r_p, annualized_v_p, var_95_p, cvar_95_p, max_drawdown_p],
                        'SPY': [annualized_r_s, annualized_v_s, var_95_s, cvar_95_s, max_drawdown_s]},
                       index=['Annualized Return', 'Annualized Vol', '95% VaR', '95% CVaR', 'Maximum Drawdown'])
performance

,Equal Weighted,SPY
Annualized Return,0.178004,0.146761
Annualized Vol,0.206661,0.209168
95% VaR,-0.017695,-0.018624
95% CVaR,-0.030997,-0.032077
Maximum Drawdown,-0.299480,-0.337173


Return: The equal-weighted portfolio boasts an annualized return of 17.80%, surpassing the slightly lower return of 14.68% for the SPY ETF. This indicates that the equal-weighted portfolio has outperformed the SPY ETF in terms of returns.

Risk: Although the annualized volatility is quite similar for both portfolios, the equal-weighted portfolio exhibits lower VaR, CVaR, Maximum drawdown, in absolute terms, suggests lower risk compared to the SPY ETF.

It's worth noting that the equal-weighted portfolio comprises the top 50 companies, while the SPY ETF tracks the S&P 500 index, representing 500 companies. This means the equal-weighted portfolio is overweight in large-cap stocks compared to the SPY ETF. During this period, mega-cap companies such as MSFT, NVDA, and META performed exceptionally well, contributing to the outperformance of the equal-weighted portfolio.

# Level 2

In [20]:
def mean_variance_optimization(returns, long_biased=False, transaction_cost=False, prev_weights = None):
    w = cp.Variable(returns.shape[1])
    risk = cp.quad_form(w, returns.cov())

    constraints = [cp.sum(w) == 1]
    if long_biased:
        constraints.append(w >= 0)

    if transaction_cost and prev_weights is not None:
        turnover = cp.norm1(w - prev_weights) / 2

        constraints.append(turnover <= 0.2)

    prob = cp.Problem(cp.Minimize(risk), constraints)
    prob.solve()

    return w.value


def monthly_rebalance(returns, method):
    return_list = []
    prev_weights = None

    for month, data in returns.resample('M'):
        if method == 'raw':
            weights = mean_variance_optimization(returns=returns.loc[:month])
        elif method == 'long_biased':
            weights = mean_variance_optimization(returns=returns.loc[:month], long_biased=True)
        elif method == 'transaction_cost':
            weights = mean_variance_optimization(returns=returns.loc[:month], long_biased=True, transaction_cost=True, prev_weights=prev_weights)
        prev_weights = weights

        portfolio_return = data.dot(weights)
        return_list.append(portfolio_return)

    optimal_return = pd.concat(return_list)
    return optimal_return

In [21]:
# Calculate mean-variance optimal portfolios
raw_portfolios = monthly_rebalance(df_returns, 'raw')
long_portfolios = monthly_rebalance(df_returns, 'long_biased')
trans_portfolios = monthly_rebalance(df_returns, 'transaction_cost')

# Calculate portfolio statistics
r_raw, v_raw, var_95_raw, cvar_95_raw, max_drawdown_raw  = portfolio_stats(raw_portfolios)
r_long, v_long, var_95_long, cvar_95_long, max_drawdown_long  = portfolio_stats(long_portfolios)
r_trans, v_trans, var_95_trans, cvar_95_trans, max_drawdown_trans  = portfolio_stats(trans_portfolios)

# Display results
mvo_performance = pd.DataFrame({'Raw MVO': [r_raw, v_raw, var_95_raw, cvar_95_raw, max_drawdown_raw],
                                'Long Biased MVO': [r_long, v_long, var_95_long, cvar_95_long, max_drawdown_long],
                                'Cost Constrained MVO': [r_trans, v_trans, var_95_trans, cvar_95_trans, max_drawdown_trans]},
                               index=['Annualized Return', 'Annualized Vol', '95% VaR', '95% CVaR', 'Maximum Drawdown'])

mvo_performance

,Raw MVO,Long Biased MVO,Cost Constrained MVO
Annualized Return,0.032762,0.059337,0.050131
Annualized Vol,0.123965,0.144795,0.155693
95% VaR,-0.012336,-0.013085,-0.013172
95% CVaR,-0.018144,-0.020978,-0.022650
Maximum Drawdown,-0.211439,-0.218830,-0.224396


Comparing the three variance-minimizing methods, all three aim to reduce volatility, resulting in lower annualized volatility compared to the equal-weighted portfolio. Additionally, both VaR and CVaR are smaller, indicating reduced potential losses.

Among the three methods, the Raw MVO exhibits the lowest risk. It achieves this by having fewer constraints compared to the other two methods, resulting in lower annualized volatility, VaR, CVaR, and maximum drawdown.

On the other hand, the Long Biased MVO achieves the highest return. While its risk falls between the other methods, it offers a better trade-off between risk and return.

It's worth noting that many high-return stocks with volatile returns, such as Nvidia and Microsoft, have smaller allocations in these portfolios. This allocation strategy contributes to the overall lower annualized return compared to the equal-weighted portfolios.

Maximum drawdowns are fairly similar across the board, showing little deviation from the equal-weighted portfolios.

In summary, each method offers a different risk-return trade-off, providing investors with options tailored to their risk tolerance and investment objectives.


# Level 3. CVaR

In [22]:
def negative_VaR(weights, returns):
    portfolio_returns = np.dot(returns, weights)
    #var = - np.percentile(portfolio_returns, 5)
    var = np.percentile(portfolio_returns, 5)
    cvar = - portfolio_returns[portfolio_returns <= var].mean()
    return cvar

def optimization_scipy(returns, method, prev_weights = None):
    n_assets = returns.shape[1]
    initial_weights = np.ones(n_assets) / n_assets
    bounds = tuple((0, 1) for asset in range(n_assets))

    if method == 'raw':
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    elif method == 'long_biased' or prev_weights is None:
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                      {'type': 'ineq', 'fun': lambda x: x})
    elif method == 'transaction_cost' and prev_weights is not None:
        turnover = lambda weights: np.linalg.norm(weights - prev_weights, ord=1) / 2
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                      {'type': 'ineq', 'fun': lambda x: x},
                      {'type': 'ineq', 'fun': lambda x: turnover(x) - 0.20})

    result = minimize(negative_VaR, initial_weights, args=(returns,),
                          method='SLSQP', bounds=bounds, constraints=constraints)
    weights_opt = result.x
    return weights_opt


def rebalance_scipy(returns, method):
    return_list = []
    prev_weights = None

    for month, data in returns.resample('M'):
        weights = optimization_scipy(returns.loc[:month], method, prev_weights)
        prev_weights = weights

        portfolio_return = data.dot(weights)
        return_list.append(portfolio_return)

    optimal_return = pd.concat(return_list)
    return optimal_return

In [23]:
# Calculate mean-variance optimal portfolios
raw_scipy = rebalance_scipy(df_returns, 'raw')
long_scipy = rebalance_scipy(df_returns, 'long_biased')
trans_scipy = rebalance_scipy(df_returns, 'transaction_cost')

# Calculate portfolio statistics
r_raw_scipy, v_raw_scipy, var_raw_scipy, cvar_raw_scipy, drawdown_raw_scipy  = portfolio_stats(raw_scipy)
r_long_scipy, v_long_scipy, var_long_scipy, cvar_long_scipy, drawdown_long_scipy  = portfolio_stats(long_scipy)
r_trans_scipy, v_trans_scipy, var_trans_scipy, cvar_trans_scipy, drawdown_trans_scipy  = portfolio_stats(trans_scipy)

# Display results
mvo_performance_scipy = pd.DataFrame({'Raw MVO': [r_raw_scipy, v_raw_scipy, var_raw_scipy, cvar_raw_scipy, drawdown_raw_scipy],
                                'Long Biased MVO': [r_long_scipy, v_long_scipy, var_long_scipy, cvar_long_scipy, drawdown_long_scipy],
                                'Cost Constrained MVO': [r_trans_scipy, v_trans_scipy, var_trans_scipy, cvar_trans_scipy, drawdown_trans_scipy]},
                               index=['Annualized Return', 'Annualized Vol', '95% VaR', '95% CVaR', 'Maximum Drawdown'])

mvo_performance_scipy

,Raw MVO,Long Biased MVO,Cost Constrained MVO
Annualized Return,0.101624,0.101599,0.098945
Annualized Vol,0.148939,0.148940,0.147639
95% VaR,-0.012740,-0.012740,-0.012606
95% CVaR,-0.020260,-0.020260,-0.019954
Maximum Drawdown,-0.186082,-0.186082,-0.179562


Comparison of Level 2 Results with Minimized CVaR:

CVaR Comparison:
Across the three methods, the CVaR values are relatively close. However, in Raw MVO, the CVaR is slightly higher, while in Cost Constrained MVO, it decreases.

Maximum Drawdown:
Significantly reduced for all three methods, aligning with the objective of minimizing left tail events and mitigating extreme losses.
Annualized return significantly higher compared to Level 2. Minimizing CVaR focuses on minimizing extreme losses while keeping the upside potential intact. This approach leads to better overall performance, as the portfolios are designed to capitalize on favorable market conditions while mitigating downside risk. Minimizing variance reduces overall portfolio volatility, impacting both upside and downside potential returns, resulting in lower annualized returns.

Comparison between Raw MVO and Long Biased MVO:
Interestingly, Raw MVO and Long Biased MVO produce very similar results. This suggests that in the case of minimizing CVaR, Raw MVO likely takes long positions predominantly. This is expected since short positions generally introduce higher downside risk.

Cost Constrained MVO:
Shows lower return and lower risk compared to the other two portfolios. This aligns with the constraints imposed, which aim to limit turnover and control transaction costs, resulting in a more conservative portfolio approach.

In summary, minimizing CVaR focuses on reducing extreme downside risk, leading to improved overall performance compared to minimizing variance. The similarities between Raw MVO and Long Biased MVO indicate their strategies, with Raw MVO likely leaning towards long positions primarily. Cost Constrained MVO portfolios show a conservative approach with lower return and risk compared to the other two portfolios.